# IAM Role

Un rol de IAM es una identidad asociada a tu cuenta de AWS que tiene políticas de permisos preconfiguradas que determinan lo que este rol puede hacer sobre los recursos de AWS. Por ejemplo, puedes definir un rol que pueda hacer *todo* con tu bucket S3,  pero nada más. Tal rol es útil si es *asumido* por un programa que sincroniza sus datos desde una máquina local en un bucket S3. Puedes estar seguro de que este programa no creará accidentalmente instancias EC2 que incurran en mayores costes. 


Los roles de IAM se utilizan para conceder permisos a los servicios de AWS para operar sobre los recursos en tu nombre. Cuando usas un servicio de AWS (por ejemplo, SageMaker), puedes definir un rol que el servicio pueda asumir en tu nombre para acceder a los recursos de AWS. Por ejemplo, el servicio SageMaker necesita acceder a cubos S3, instancias EC2, Registros Elásticos de Contenedores, etc. Para evitar incurrir en demasiados costes de computación, puede definir un rol que sea capaz de crear sólo instancias EC2 de bajo coste. Cuando SageMaker asume este rol y ejecuta su flujo de trabajo de ML, puede estimar el límite superior del costo de cómputo basado en la política de EC2 del rol. 

Para lecturas más extensas sobre el rol IAM, consulte la [documentación de AWS](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_terms-and-concepts.html#iam-term-role-chaining)

## Entornos para ejecutar este notebook

1. Si estás ejecutando este noebook desde una instancia de EC2, asegúrate de que la variable de entorno `AWS_PROFILE` está establecida en `default`.

2. Si estás ejecutando este notebook en tu máquina local, tendrás que instalar y configurar la interfaz de línea de comandos de aws. Siga [este enlace](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) para hacerlo.

El usuario/rol IAM que ejecute este notebook debe tener al menos el permiso `iam:CreateRole` y `iam:DeleteRole` y `sts:AssumeRole`. 

No recomiendo ejecutarlo en una Instancia de SageMaker Notebook o Studio, porque el rol que utiliza dicho servicio tiene de serie permisos mucho más amplios que los que vamos a demostrar en este notebook para demostrar cómo se puede crear un rol IAM como un usuario IAM a través de AWS Python SDK (boto3). 

## boto3, primera aproximación

https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

### Control de errores

https://docs.python.org/3/tutorial/errors.html

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/error-handling.html

In [ ]:
import botocore.exceptions

for key, value in sorted(botocore.exceptions.__dict__.items()):
    if isinstance(value, type):
        print(key)

## Crear un rol IAM

Cuando se crea un rol IAM es necesario especificar

1. Qué entidades de AWS (usuarios o servicios) pueden asumir este rol. Está definido en una *política de confianza*.
2. Qué permisos tiene este rol. Está definido por una *política de permisos*.

Para más detalles, consulta la sección [Creación de un rol para delegar permisos a un servicio de AWS](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_for-service.html) de la documentación oficial. 

La entidad a la que asume el rol se denomina *Principal*. 

In [ ]:
import boto3
import pprint
import json
import time

pp = pprint.PrettyPrinter(indent=1)
iam = boto3.client("iam")

En primer lugar vamos a conseguir el ARN de nuestro usuario, el que hemos configurado al disponibilizar nuestro juego de claves

In [ ]:
caller_arn = boto3.client("sts").get_caller_identity()["Arn"]

print(caller_arn)

In [ ]:
def create_execution_role(role_name="basic-role"):
    """Crear un rol de servicio para adquirir servicios en su nombre


    Args:
        role_name (str): nombre del rol

    Return:
        dict
    """

    # si el rol ya existe, se eliminará

    # Nota: debes asegurarte de que el rol no se
    # usa en producción, porque este código va a
    # borrar el rol y crear uno nuevo

    def find_role(role_res, role_name):
        for r in role_res["Roles"]:
            if r["RoleName"] == role_name:
                return True
        return False

    def delete_role(role_res, role_name):
        if find_role(role_res, role_name):
            role = boto3.resource("iam").Role(role_name)
            for p in role.attached_policies.all():
                role.detach_policy(PolicyArn=p.arn)

            iam.delete_role(RoleName=role.name)
            return

    role_res = iam.list_roles(MaxItems=10)
    delete_role(role_res, role_name)

    # Gestionamos la paginación
    while "Marker" in role_res:
        role_res = iam.list_roles(MaxItems=10, Marker=role_res["Marker"])
        delete_role(role_res, role_name)

    # Trust policy document
    trust_relation_policy_doc = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "AWS": caller_arn,  # Permite que el caller asuma este rol
                    "Service": ["sagemaker.amazonaws.com"],  # Permite al servicio SageMaker asumir el rol
                },
                "Action": "sts:AssumeRole",
            }
        ],
    }

    res = iam.create_role(
        RoleName=role_name, AssumeRolePolicyDocument=json.dumps(trust_relation_policy_doc)
    )
    return res

La política de confianza anterior dice que confiamos al usuario de la sesión actual de boto3 (nosotros mismos) y a SageMaker para que asuman este rol. 

Creamos el rol utilizando la función definida arriba

In [ ]:
role_res = create_execution_role()
pp.pprint(role_res)

Ahora, vamos a dar al rol algunos permisos. El diccionario de abajo es un ejemplo de permiso de política. Dice: permitir que el rol liste buckets en la cuenta de AWS.

In [ ]:
basic_s3_permission = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:List*"  # Permite al rol la acción de listar, p.ej acceso de lectura
                # "s3:*" # Permite cualquier tipo de acción sobre s3
            ],
            "Resource": ["arn:aws:s3:::*"],
        }
    ],
}

Crear la policy

Si la política ya esxiste entonces hay que borrarla primero

Nota: debes asegurarte de que no tienes una política el mismo nombre en producción, porque la vamos a borrar
y crearemos una nueva con el documento anterior

In [ ]:
def attach_permission(role_name, policy_name, policy_doc):
    """Attach a basic permission policy to the role"""

    policy = None
    for p in iam.list_policies()["Policies"]:
        if p["PolicyName"] == policy_name:
            # Antes de borrar la policy, necesitamos desasociarla
            # de todas las entidades a las que esté asociada
            policy = boto3.resource("iam").Policy(p["Arn"])

            # 1. Desasociamos de todos los grupos
            for grp in policy.attached_groups.all():
                policy.detach_group(GroupName=grp.name)

            # 2. Desasociamos de todos los usuarios
            for usr in policy.attached_users.all():
                policy.detach_user(UserName=usr.name)

            # 3. Desasociamos de todos los roles
            for rol in policy.attached_roles.all():
                policy.detach_role(RoleName=rol.name)

            break

    if policy is not None:
        iam.delete_policy(PolicyArn=policy.arn)

    # Creamos una nueva política
    policy = iam.create_policy(PolicyName=policy_name, PolicyDocument=json.dumps(policy_doc))[
        "Policy"
    ]

    # Asociamos la policy con el rol
    res = iam.attach_role_policy(RoleName=role_name, PolicyArn=policy["Arn"])
    return res

Con la siguiente ejecución vamos a asociar ambas entidades y a examinar la respuesta

In [ ]:
perm_res = attach_permission(
    role_name=role_res["Role"]["RoleName"],
    policy_name="BasicS3Policy",
    policy_doc=basic_s3_permission,
)

pp.pprint(perm_res)

Esperamos 15 segundos para que la policý se propague

In [ ]:
time.sleep(15)

## Testeamos el rol

Ahora podemos verificar que el rol que acabamos de crear (`basic-role`) tiene permiso para listar todos los buckets de S3 en nuestra cuenta y no tiene permiso para hacer nada más con ningún otro recurso de AWS. 

Creamos una nueva sesión de boto3 asumiendo el rol

In [ ]:
import time

now = str(time.time()).split(".")[0]

obj = boto3.client("sts").assume_role(RoleArn=role_res["Role"]["Arn"], RoleSessionName=now)

cred = obj["Credentials"]

Vamos a observar las credenciales generadas. Recuerda borrar la ejecución de esta celda para no subir credenciales a un repositorio

In [ ]:
# print(creds)

Generamos un objeto de tipo sesión utilizando las credenciales anteriores

In [ ]:
sess = boto3.session.Session(
    aws_access_key_id=cred["AccessKeyId"],
    aws_secret_access_key=cred["SecretAccessKey"],
    aws_session_token=cred["SessionToken"],
)

Iniciamos un nuevo cliente de S3 utilizando la sesión generada

In [ ]:
s3 = sess.client("s3")

Listamos los buckets, ya que hemos asignado permisos de lectura

In [ ]:
pp.pprint(s3.list_buckets())

Intentemos crear un bucket con el cliente S3

Se espera que falle, porque el `basic-role` no tiene asignada ninguna policy para crear buckets

Recordemos que el nombre del bucket tiene que ser único. ¿Por qué?

In [ ]:
import time


def create_bucket(s3_client):
    try:
        now = str(time.time()).split(".")[0]
        res = s3_client.create_bucket(
            Bucket="bucket-{}".format(now),  
            #CreateBucketConfiguration={"LocationConstraint": sess.region_name}, # not needed in us-east-1 https://stackoverflow.com/questions/51912072/invalidlocationconstraint-error-while-creating-s3-bucket-when-the-used-command-i
        )

        return res
    except Exception as e:
        print(e)
        return e


# Va a fallar!
create_bucket(s3)

Ahora, solucionemos esto para que el rol `basic-role` pueda crear buckets

Vamos a generar una nueva policy

In [ ]:
create_bucket_permission = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["s3:CreateBucket"], "Resource": ["arn:aws:s3:::*"]}
    ],
}

Nota: las políticas asociadas son incrementales, esto significa que si adjuntamos `create_bucket_permission` a `basic-role`, el efecto de `basic_s3_permission` sigue vigente.

Por lo tanto, `basic-role` sigue siendo capaz de realizar acciones de `List*`. 

In [ ]:
perm_res = attach_permission(
    role_name=role_res["Role"]["RoleName"],
    policy_name="CreateBucket",
    policy_doc=create_bucket_permission,
)

Observamos la response

In [ ]:
pp.pprint(perm_res)
time.sleep(15)

Seguimos utilizando la sesión del rol que hemos asumido, por lo que no es necesario crear un nuevo token

In [ ]:
res = create_bucket(s3)
location = res["Location"]
bucket_name = location.split("/")[1].split(".")[0]

Ahora, vamos a eliminar el bucket que acabamos de crear.

En primer lugar concedemos a `basic-role` el permiso para eliminar el bucket. 

In [ ]:
delete_bucket_permission = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["s3:DeleteBucket"], "Resource": ["arn:aws:s3:::*"]}
    ],
}

perm_res = attach_permission(
    role_name=role_res["Role"]["RoleName"],
    policy_name="DeleteBucket",
    policy_doc=delete_bucket_permission,
)

Observamos la respuesta

In [ ]:
res = s3.delete_bucket(Bucket=bucket_name)
pp.pprint(res)

## Políticas gestionadas por AWS

Hasta ahora hemos aprendido a crear un rol de IAM y a concederle permisos sobre recursos de AWS a través de las políticas de IAM. El ejemplo que has visto arriba hemos utilizado buckets de S3 como servicio de ejemplo, pero la misma idea se puede generalizar cuando necesites definir políticas más complicadas en más servicios. 

No siempre es necesario definir tus propias políticas. Amazon gestiona una lista de políticas de uso común. Puedes verlas a través de la [consola](https://console.aws.amazon.com/iam/home?region=us-west-2#/policies) o dentro de este notebook llamando a la API `ListPolicies` del servicio IAM.

Listamos alguna las políticas relacionadas con S3 gestionadas por AWS (recuerda que la salida nos llega paginada)

In [ ]:
for p in iam.list_policies(Scope="AWS")["Policies"]:
    if "S3" in p["PolicyName"]:
        pp.pprint(p)
        print("=" * 80)

Una política de S3 mantenida por Amazon se llama `AmazonS3FullAccess`. Como es de esperar, esta política otorga permisos para realizar todas las acciones posibles sobre recursos S3. 
Análogamente, la política `AmazonEC2FullAccess` otorga el rol para realizar todas las acciones en sus recursos EC2.
`AmazonSageMakerFullAccess` proporciona acceso completo a Amazon SageMaker a través de la consola de administración de AWS y el SDK. También proporciona acceso a los servicios relacionados.

Las políticas están versionadas. Esto permite de ajustar iterativamente los permisos a un rol de ejecución sin cambiar el nombre de la política. 

Vamos a ver con más detalle la política `AmazonSageMakerFullAccess`

In [ ]:
sagemaker_full = iam.get_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess")

In [ ]:
versions = iam.list_policy_versions(PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess")
pp.pprint(versions["Versions"][0])

Podemos ver que la última versión publicada de la política es `v19`. 
The latest version for `AmazonSageMakerFullAccess` policy is 

Podemos hacer una query para observar una versión específica de la política indicando el `VersionID`

In [ ]:
pp.pprint(
    iam.get_policy_version(
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess", VersionId="v19"
    )
)

Como podemos ver,para la mayoría de los casos de uso de Amazon SageMaker, la política `AmazonSageMakerFullAccess` sería suficiente.

Vamos a asociar la política `AmazonSageMakerFullAccess` al rol `basic-role`

In [ ]:
res = iam.attach_role_policy(
    RoleName=role_res["Role"]["RoleName"],
    PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
)

Observamos la respuesta

In [ ]:
pp.pprint(res)

## Verificación de que un rol de ejecución tiene suficientes permisos

Cuando se utiliza una política "paragüas" como `AmazonSageMakerFullAccess` a veces es útil comprobar que tiene suficientes permisos para incluir todas las acciones que nuestro rol de ejecución necesitará sobre los recursos de AWS. Esto se puede hacer llamando a la API `SimulatePrincipalPolicy` desde el servicio IAM.

Esta API recibe a la entidad de la política (usuario o rol de IAM) y una lista de acciones, los recursos sobre los que se realizan las acciones, y nos informa si la entidad que ostenta de la política tiene el permiso para hacer dichas acciones. 

El rol de ejecución de SageMaker necesita interactuar con recursos de Elastic Container Registry (ECR). Vamos a ver lo que su rol de ejecución puede hacer sobre ECR cuando se aocia con la política `AmazonSageMakerFullAccess`. 

In [ ]:
sm_policy_res = iam.simulate_principal_policy(
    PolicySourceArn=role_res["Role"]["Arn"],  # policy carrier (my execution role)
    ActionNames=["ecr:GetAuthorizationToken"]  # I want to know if my execution role can get
    # authorization token from my ECR
)

Observamos la respuesta

In [ ]:
pp.pprint(sm_policy_res)

La respuesta dice que el rol de ejecución tiene permiso para obtener el token de autorización del ECR. 

Ahora vamos a comprobar si podemos hacer cualquier tipo de acción sobre S3

In [ ]:
sm_policy_res = iam.simulate_principal_policy(
    PolicySourceArn=role_res["Role"]["Arn"],
    ActionNames=["s3:*"]
)

Observamos la respuesta

In [ ]:
pp.pprint(sm_policy_res)

Esto quiere decir que el rol de ejecución no puede hacer todo sobre recursos S3.

Limitemos el alcance de las acciones y recursos de S3. Solamente queremos ejecutar las acciones:

- GetObject
- PutObject
- DeleteObject
- AbortMultipartUpload

Y solamente sobre buckets S3 que cumplan el patrón `*sagemaker*`

In [ ]:
sm_policy_res = iam.simulate_principal_policy(
    PolicySourceArn=role_res["Role"]["Arn"],  # policy carrier (my execution role)
    ActionNames=["s3:GetObject", "s3:PutObject", "s3:DeleteObject", "s3:AbortMultipartUpload"],
    ResourceArns=["arn:aws:s3:::*sagemaker*"],  # bucket names with sagemaker in it
)

Observamos la respuesta

In [ ]:
pp.pprint(sm_policy_res)

Esto dice que nuestro rol de ejecución puede realizar 

-`s3:GetObject`
-`s3:PutObject`
-`s3:DeleteObject`
-`s3:AbortMultipartUpload` 

en nuestro bucket S3, que incluye `sagemaker` en el nombre del bucket. Echa un vistazo al documento de política `AmazonSageMakerFullAccess` para ver más detalles. 

## Clean up

Si solo piensas utilizar el rol para ejecutar cierta aplicación una vez, entonces es una buena práctica eliminar el rol. Si planea eliminar un rol, asegúrese de **haber eliminado todas las políticas asociadas a él** y de **no tener ninguna instancia de Amazon EC2 en ejecución con el rol que va a eliminar. La eliminación de un rol o un perfil de instancia que esté asociado a una instancia en ejecución interrumpirá cualquier aplicación que se esté ejecutando en la instancia.

In [ ]:
# detach attached policies
attached_policies = iam.list_attached_role_policies(RoleName="basic-role")["AttachedPolicies"]
for p in attached_policies:
    iam.detach_role_policy(RoleName=role_res["Role"]["RoleName"], PolicyArn=p["PolicyArn"])

# delete the role
res = iam.delete_role(RoleName=role_res["Role"]["RoleName"])

pp.pprint(res)